In [0]:
!pip install seqeval==0.0.5
!pip install pytorch_pretrained_bert==0.4.0

import os

# import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import pandas as pd

import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
import subprocess
from subprocess import check_output

In [0]:
def getStringFromDataFile(fname):
    dataFilePath    = fname
    dataFile        = open(dataFilePath, "r", encoding='utf-8')
    dataStr         = dataFile.read()
    # print(dataStr)
    return dataStr


def generate_dataset_part_a(outname):
    out = getStringFromDataFile(outname)
    out = out.split("<DATADEL>")  
    data = []
    for x in out:
        data.append(x.split("<XYDEL>"))
    dataset=[]
    for d in data:
        temp = []
        temp.append(d[0].split(" "))
        temp.append(d[1].split(" "))
        dataset.append(temp)
    return dataset

data = generate_dataset_part_a("train_data")
x_train = [x[0] for x in data]
y_train = [x[1] for x in data]
print("size of data: " + str(len(x_train)))
print("-----------------------------------------------------")
print(x_train[0])
print("-----------------------------------------------------")
print(y_train[0])

size of data: 4036
-----------------------------------------------------
['השימוש', 'הראשון', 'המוכר', 'במונח', 'קפיטליזם', 'מופיע', 'אצל', 'תאקרי', 'בשנת', '0', 'במשמעות', 'בעלות', 'על', 'הון', 'השימוש', 'המודרני', 'הרחב', 'הראשון', 'במונח', 'בא', 'בספר', 'הקפיטל', 'של', 'קרל', 'מרקס', 'ופרידריך', 'אנגלס', 'המתייחס', 'למערכת', 'הייצור', 'הקפיטליסטית', 'ול', 'קפיטליסט', 'במובן', 'של', 'בעלים', 'של', 'אמצעי', 'הייצור', 'שימושם', 'המקורי', 'של', 'מרקס', 'ואנגלס', 'במונח', 'היה', 'ככינוי', 'גנאי', 'מאקס', 'ובר', 'השתמש', 'בו', 'בחיבורו', 'הידוע', 'האתיקה', 'הפרוטסטנטית', 'ורוח', 'הקפיטליזם', 'אך', 'רק', 'לאחר', 'פרסום', 'ספרו', 'של', 'הכלכלן', 'ורנר', 'זומברט', 'קפיטליזם', 'מודרני', 'משנת', '0', 'המונח', 'הפך', 'לתיאור', 'תפיסה', 'המנוגדת', 'לסוציאליזם', 'כיום', 'אף', 'שאדם', 'סמית', 'לא', 'השתמש', 'מעולם', 'במונח', 'קפיטליזם', 'ותיאר', 'את', 'המערכת', 'הכלכלית', 'המועדפת', 'עליו', 'כ', 'מערכת', 'החירות', 'הטבעית', 'יש', 'הרואים', 'בו', 'אבי', 'הקפיטליזם']
--------------------------------

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

def tokenize(sentences, orig_labels):
    tokenized_texts = []
    labels = []
    for sent, sent_labels in zip(sentences, orig_labels):
        bert_tokens = []
        bert_labels = []
        for orig_token, orig_label in zip(sent, sent_labels):
            b_tokens = tokenizer.tokenize(orig_token)
            bert_tokens.extend(b_tokens)
            for b_token in b_tokens:
                bert_labels.append(orig_label)
        tokenized_texts.append(bert_tokens)
        labels.append(bert_labels)

        assert len(bert_tokens) == len(bert_labels)

    return tokenized_texts, labels


train_tokenized_texts, train_labels = tokenize(x_train, y_train)
print(train_tokenized_texts[0])
print(train_labels[0])

['השימוש', 'הראשון', 'ה', '##מו', '##כר', 'ב', '##מו', '##נח', 'ק', '##פי', '##טל', '##יז', '##ם', 'מופיע', 'אצל', 'ת', '##אק', '##רי', 'בשנת', '0', 'ב', '##מש', '##מעות', 'בעלות', 'על', 'ה', '##ון', 'השימוש', 'ה', '##מוד', '##רני', 'הר', '##חב', 'הראשון', 'ב', '##מו', '##נח', 'ב', '##א', 'בספר', 'ה', '##ק', '##פי', '##טל', 'של', 'ק', '##רל', 'מ', '##רק', '##ס', 'ו', '##פר', '##יד', '##ריך', 'אנג', '##לס', 'ה', '##מת', '##יי', '##חס', 'ל', '##מערכת', 'ה', '##יי', '##צור', 'ה', '##ק', '##פי', '##טל', '##יס', '##טית', 'ו', '##ל', 'ק', '##פי', '##טל', '##יס', '##ט', 'ב', '##מו', '##בן', 'של', 'בעלי', '##ם', 'של', 'א', '##מ', '##צע', '##י', 'ה', '##יי', '##צור', 'שימוש', '##ם', 'המקורי', 'של', 'מ', '##רק', '##ס', 'ו', '##א', '##נג', '##לס', 'ב', '##מו', '##נח', 'היה', 'כ', '##כי', '##נוי', 'ג', '##נאי', 'מ', '##אק', '##ס', 'וב', '##ר', 'ה', '##שתמש', 'בו', 'ב', '##חי', '##בור', '##ו', 'ה', '##ידוע', 'ה', '##את', '##יקה', 'ה', '##פר', '##וט', '##סט', '##נטית', 'ו', '##רו', '##ח', 'ה', '##ק'

In [0]:
tags_vals = ['none', ',','.','!','?']
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals)}

MAX_LEN = 200 ####TO_DO
bs = 32 ####TO_DO_TOO

def pad_sentences_and_labels(tokenized_texts, labels):
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                              maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                         maxlen=MAX_LEN, value=tag2idx["none"], padding="post",
                         dtype="long", 
                         truncating="post")

    attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
    
    return input_ids, tags, attention_masks
  

input_ids, tags, attention_masks = pad_sentences_and_labels(train_tokenized_texts, train_labels)


In [0]:
tr_inputs = torch.tensor(input_ids)
tr_tags = torch.tensor(tags)
tr_masks = torch.tensor(attention_masks)

train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

print("Device: " + str(device))
print("Number of gpus: " + str(n_gpu))
if torch.cuda.is_available():
    print("Name of gpu: " + torch.cuda.get_device_name(0))

Device: cuda
Number of gpus: 1
Name of gpu: Tesla T4


In [0]:
model = BertForTokenClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=len(tag2idx))

if torch.cuda.is_available():
    model.cuda()


FULL_FINETUNING = False
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


epochs = 50
max_grad_norm = 1.0


In [0]:
for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,
                     labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss / nb_tr_steps))

Epoch:   2%|▏         | 1/50 [02:53<2:21:29, 173.25s/it]

Train loss: 1.4419150408797377


Epoch:   4%|▍         | 2/50 [05:45<2:18:17, 172.87s/it]

Train loss: 0.9609540672752801


Epoch:   6%|▌         | 3/50 [08:37<2:15:10, 172.56s/it]

...

Train loss: 0.33479553953869134


Epoch:  98%|█████████▊| 49/50 [2:20:10<02:51, 171.52s/it]

Train loss: 0.3345003496474168


Epoch: 100%|██████████| 50/50 [2:23:02<00:00, 171.63s/it]

Train loss: 0.33491588436712433


In [0]:



def generate_dataset_part_a(outname):
    out = getStringFromDataFile(outname)
    out = out.split("<DATADEL>")  
    data = []
    for x in out:
        data.append(x.split("<XYDEL>"))
    dataset=[]
    for d in data:
        temp = []
        temp.append(d[0].split(" "))
        temp.append(d[1].split(" "))
        dataset.append(temp)
    return dataset

data = generate_dataset_part_a("test_data")
x_test = [x[0] for x in data]
y_test = [x[1] for x in data]

print("-----------------------------------------------------")
print(x_test[0])
print("-----------------------------------------------------")
print(y_test[0])

-----------------------------------------------------
['פילוסופיה', 'של', 'המדע', 'הוא', 'ענף', 'בפילוסופיה', 'העוסק', 'ביסודות', 'הנחות', 'ובמשמעויות', 'הפילוסופיות', 'של', 'המדעים', 'ענף', 'מגוון', 'זה', 'מתייחס', 'לכל', 'המדעים', 'מדעי', 'הטבע', 'כגון', 'פיזיקה', 'וביולוגיה', 'מדעי', 'החברה', 'כגון', 'פסיכולוגיה', 'וכלכלה', 'ומדעי', 'הרוח', 'יש', 'הרואים', 'בפילוסופיה', 'של', 'המדע', 'אפיסטמולוגיה', 'ואף', 'מטאפיזיקה', 'של', 'המדע', 'הממשי', 'הפילוסופיה', 'של', 'המדע', 'שואפת', 'לענות', 'ולהסביר', 'שאלות', 'כגון', 'טבעם', 'של', 'קביעות', 'ומושגים', 'מדעיים', 'האופן', 'שבו', 'הם', 'נוצרים', 'כיצד', 'המדע', 'מסביר', 'חוזה', 'ומשתמש', 'בטבע', 'באמצעות', 'טכנולוגיה', 'כיצד', 'אפשר', 'להחליט', 'על', 'הדיוק', 'של', 'מידע', 'הניסוח', 'והשימוש', 'במתודות', 'מדעיות', 'אופני', 'החשיבה', 'שבהם', 'משתמשים', 'על', 'מנת', 'להגיע', 'למסקנות', 'והמשמעויות', 'של', 'המתודות', 'והמודלים', 'המדעיים', 'לחברה', 'ככלל', 'ולמדעים', 'עצמם']
-----------------------------------------------------
['none', 'non

In [0]:
  import pandas as pd
  
  def simplify_label(label):
    return label
  
  classes_without_O = [",",".","?","!"]

  test_tokenized_texts, test_labels = tokenize(x_test, y_test)
  input_ids, tags, attention_masks = pad_sentences_and_labels(test_tokenized_texts, test_labels)

  val_inputs = torch.tensor(input_ids)
  val_tags = torch.tensor(tags)
  val_masks = torch.tensor(attention_masks)

  test_data = TensorDataset(val_inputs, val_masks, val_tags)
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)
  
  
  model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  predictions, true_labels = [], []
  counter = 0
  for batch in test_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch

      with torch.no_grad():
          tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                attention_mask=b_input_mask, labels=b_labels)
          logits = model(b_input_ids, token_type_ids=None,
                         attention_mask=b_input_mask)
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      true_labels.append(label_ids)

      tmp_eval_accuracy = flat_accuracy(logits, label_ids)

      eval_loss += tmp_eval_loss.mean().item()
      eval_accuracy += tmp_eval_accuracy

      nb_eval_examples += b_input_ids.size(0)
      nb_eval_steps += 1
  eval_loss = eval_loss / nb_eval_steps
  print("Validation loss: {}".format(eval_loss))
  print("Validation Accuracy: {}".format(eval_accuracy / nb_eval_steps))
  pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
  test_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
  print("F1-Score: {}".format(f1_score(pred_tags, test_tags)))

  y_true = pd.Series(test_tags)
  y_pred = pd.Series(pred_tags)
  cross_tab = pd.crosstab(y_true, y_pred, rownames=['Real Label'], colnames=['Prediction'], margins=True)
  report = classification_report(y_true, y_pred, labels=classes_without_O, target_names=classes_without_O)
  report_with_O = classification_report(y_true, y_pred)

  t_y_true = [simplify_label(tags_vals[p_i]) for p in predictions for p_i in p]
  t_y_pred = [simplify_label(tags_vals[l_ii]) for l in true_labels for l_i in l for l_ii in l_i]
  
  for i in range(2200):
    if (t_y_pred[i]!= "none"):
      print(str(i) + " - " + t_y_true[i] +" "+ t_y_pred[i])
 
  
  transformed_y_true = pd.Series(t_y_true)
  transformed_y_pred = pd.Series(t_y_pred)

  cross_tab_transformed = pd.crosstab(transformed_y_true, transformed_y_pred, rownames=['Real Label'], colnames=['Prediction'], margins=True)
  report_transformed = classification_report(transformed_y_true, transformed_y_pred, labels=classes_without_O, target_names=classes_without_O)
  report_with_O_transformed = classification_report(transformed_y_true, transformed_y_pred)

Validation loss: 0.304429829120636
Validation Accuracy: 0.89375
F1-Score: 0.022963367960634227


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


18 - none ,
19 - none ,
20 - none ,
33 - none .
34 - none .
35 - . .
45 - none ,
46 - none ,
47 - none ,
49 - none ,
50 - none ,
55 - none ,
56 - none ,
57 - none ,
59 - none ,
65 - none ,
66 - none ,
67 - none ,
71 - none .
72 - . .
99 - none .
100 - none .
101 - none .
102 - . .
121 - none ,
122 - none ,
123 - none ,
150 - none ,
151 - none ,
152 - none ,
153 - none ,
213 - none ,
214 - none ,
215 - none ,
216 - none ,
217 - none ,
218 - none ,
219 - none ,
224 - none .
225 - none .
246 - . .
248 - none ,
260 - none .
261 - none .
262 - none .
263 - none .
264 - none .
274 - none ,
318 - none .
432 - . .
433 - . .
460 - none .
512 - none .
513 - none .
514 - none .
515 - none .
540 - none ,
541 - none ,
542 - none ,
548 - none .
549 - none .
550 - none .
551 - none .
552 - none .
553 - none .
554 - none ,
556 - none ,
557 - none ,
558 - none ,
597 - none ,
598 - none ,
599 - . ,
611 - none ,
624 - none .
625 - none .
626 - none .
627 - none .
628 - none .
629 - none .
656 - none .
68

In [0]:
print(report)

              precision    recall  f1-score   support

           ,       0.47      0.04      0.08      1397
           .       0.63      0.15      0.24       882
           ?       0.00      0.00      0.00         0
           !       0.00      0.00      0.00         2

   micro avg       0.57      0.08      0.14      2281
   macro avg       0.27      0.05      0.08      2281
weighted avg       0.53      0.08      0.14      2281



In [0]:
print(report_with_O)

              precision    recall  f1-score   support

           !       0.00      0.00      0.00         2
           ,       0.47      0.04      0.08      1397
           .       0.63      0.15      0.24       882
        none       0.89      1.00      0.94     17319

   micro avg       0.89      0.89      0.89     19600
   macro avg       0.50      0.30      0.31     19600
weighted avg       0.85      0.89      0.85     19600

